In [1]:
import torch
import torchvision

import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

# Ví dụ về Autograd

In [2]:
# Xây dựng đồ thị tính toán với phương trình: y = 5*x + 10
x = torch.tensor(5., requires_grad=True)
w = torch.tensor(3., requires_grad=True)
b = torch.tensor(10., requires_grad=True)

y = w*x + b

# Thực hiện tính gradients.
y.backward()

print("x.grad: ", x.grad)
print("w.grad: ", w.grad)
print("b.grad: ", b.grad)

x.grad:  tensor(3.)
w.grad:  tensor(5.)
b.grad:  tensor(1.)


# Ví dụ về Autograd (tiếp theo)

In [4]:
# Tạo 2 tensor x, y có kích thước lần lượt là (10, 4) và (10, 3)
x = torch.randn(10, 4)
y = torch.randn(10, 3)

# Tạo fully connected layer
linear_layer = nn.Linear(4, 3)
print("w: ", linear_layer.weight)
print("b: ", linear_layer.bias)

# Xây dựng hàm loss và optimizer 
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear_layer.parameters(), lr=0.001)

# Lan truyền tiến 
forward = linear_layer(x)

# Tính giá trị loss
loss = criterion(forward, y)
print("loss: ", loss.item())

# Lan truyền ngược
loss.backward()

# Xuất giá trị đạo hàm
print("dL/dW: ", linear_layer.weight.grad)
print("dL/dB: ", linear_layer.bias.grad)

optimizer.step()

# Xuất kết quả sau một bước đaọ hàm
pred = linear_layer(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[ 0.3590, -0.4424,  0.2856, -0.2533],
        [-0.0184, -0.4492,  0.0044,  0.2750],
        [-0.4786, -0.3913,  0.3261,  0.2091]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.4702, -0.2477,  0.4032], requires_grad=True)
loss:  1.0402820110321045
dL/dW:  tensor([[-0.1518, -0.1619,  0.3419, -0.3265],
        [-0.0149,  0.2259, -0.2498,  0.3257],
        [-0.1852, -0.1724,  0.1863,  0.0063]])
dL/dB:  tensor([0.1047, 0.0153, 0.4232])
loss after 1 step optimization:  1.0395008325576782


# Load dữ liệu với numpy

In [5]:
x = np.array([[1, 2], [3, 4]])

# Chuyển đổi numpy array sang torch tensor
y = torch.from_numpy(x)
print("type(y): ", type(y))

# Chuyển đổi torch tensor sang numpy array
z = y.numpy()
print("type(z): ", type(z))

type(y):  <class 'torch.Tensor'>
type(z):  <class 'numpy.ndarray'>


# Input pipeline 

In [6]:
# Sử dụng dữ liệu với torchvision.datasets
train_dataset = torchvision.datasets.CIFAR10(root="./data",
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

image, label = train_dataset[0]
print("size: ", image.size())
print("labelL: ", label)

# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)

# Khi bắt đầu lặp, sẽ lần lượt đọc dữ liệu đưa vào queue và thread
data_iter = iter(train_loader)

# Thực hiện mini-batch trên các ảnh và nhãn tương ứng
images, labels = data_iter.next()

# Quá trình load ảnh thực tế được thực hiện như sau
for image, label in train_loader:
  # Quá trình huấn luyện được thực hiện tại đây
  pass

100%|██████████| 170M/170M [14:01<00:00, 203kB/s]  


size:  torch.Size([3, 32, 32])
labelL:  6


AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'

# Input pipeline for custom dataset


In [ ]:
# Xây dựng hàm xử lý cho custom dataset
class CustomDataset(object): # torch.utils.data.Dataset
  def __init__(self):
    # Khởi tạo danh sách đường dẫn hình hoặc tên hình.
    pass
  
  def get_item(self, index):
    # Đọc data từ file (numpy.fromfile, PIL.Image.open)
    # Tiền xử lý dữ liệ (torchvision.Transform)
    # Trả về một cặp dữ liệu (image và label)
    pass 
  
  def __len__(self):
    # Trả về số lượng mẫu trong dữ liệu 
    pass 


# Pretrained model

In [5]:
# Sử dụng pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# Finetune model 
for param in resnet.parameters():
  param.requires_grad = False

# Thay thế top k layer đầu tiên cho finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # k = 100

# Lan truyền tiến 
image = torch.rand(64, 3, 224, 224)
outputs = resnet(image)
print("output size: ", outputs.size())

output size:  torch.Size([64, 100])


# Save and Load model

In [6]:
torch.save(resnet, "model_resnet18.ckpt")
model = torch.load('model_resnet18.ckpt')
print("load model....")

# Save and load only the model parameters (thường dùng).
torch.save(resnet.state_dict(), 'params_resnet.ckpt')
resnet.load_state_dict(torch.load('params_resnet.ckpt'))
print("load params......")

load model....
load params......
